## Claude.ai

This is just some messing around with the `example_hists` files.

### Docs:

* in theory, there is documentation here: https://docs.anthropic.com/en/home, 
* but in reality I just asked "Hey Claude.  How do I use your API through Python to upload an image and ask you questions about it?" followed by "Is there anyway to set the context?  For example, in chatgpt you have the ""role": "system", "content"" part of your message where you would say things like "You are a helpful assistant in charge of automating a process".  Or does one just incorporate that into the "question" part of the inputs?" and starting building from there

#### Key Points (cp-claude)

* API Key: Get your API key from the [Anthropic Console](https://console.anthropic.com/) and set it as an environment variable or pass it directly
* Supported formats: JPEG, PNG, GIF, and WebP images
* Model: Use `claude-sonnet-4-20250514` for Claude Sonnet 4
* Size limits: Images should be under 5MB and no larger than 8000x8000 pixels



First, install anthropic api (also, see .yml file for the environment for this project)

In [ ]:
#!pip install anthropic

Where are things stored/going to be stored?

In [ ]:
dir_api = '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/LLM_outputs/claude_api/' #store API results for example-hists

key_file = '/Users/jnaiman/.claudeai/key.txt'

jsons_dir = '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/' # directory where jsons created with figure are stored
imgs_dir = '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/imgs/' # where images are stored

# for saving temp images for reading in
tmp_dir = '/Users/jnaiman/Downloads/tmp/'

img_format = 'jpeg'

In [ ]:
import anthropic
import base64
from PIL import Image
import numpy as np
import json
import re
import pickle
import os
from glob import glob

from sys import path
path.append('../')
from utils.llm_utils import parse_qa, load_image

In [7]:
# setup
with open(key_file,'r') as f:
    api_key = f.read()

client = anthropic.Anthropic(
  api_key=api_key.strip(),  # this is also the default, it can be omitted
)

In [8]:
jsons_to_parse = glob(jsons_dir + '/*.json')
jsons_to_parse[:3]

['/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/nclust_3_trial9.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/nclust_5_trial3.json',
 '/Users/jnaiman/LLM_VQA_JCDL2025/example_hists/jsons/nclust_2_trial0.json']